In [1]:
%%capture --no-stderr
%pip install -qU langgraph
%pip install -qU langchain_ollama
%pip install -qU langchain-openai
%pip install -qU typing_extensions
%pip install -qU requests
%pip install -qU panel
%pip install -qU jupyter_bokeh
%pip install -qU html2text

In [2]:
# import os

# os.environ["OPENAI_API_VERSION"] = "YOUR_OPENAI_API_VERSION"
# os.environ["AZURE_OPENAI_ENDPOINT"] = "YOUR_AZURE_OPENAI_ENDPOINT"
# os.environ["AZURE_OPENAI_API_KEY"] = "YOUR_AZURE_OPENAI_API_KEY"

In [3]:
# Step x. define the state
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

class State(TypedDict):
    num_called: int
    messages: Annotated[list, add_messages]

In [4]:
# Step x. define the graph builder
from langgraph.graph import StateGraph
graph_builder = StateGraph(State)

In [5]:
# Import Azure OpenAI
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment="gpt-4",
    api_version="2023-03-15-preview",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # organization="...",
    # model="gpt-35-turbo",
    # model_version="0125",
    # other params...
)

In [6]:
# TOOLS (4) ==================>
from langchain_core.tools import tool
import requests
import html2text

@tool
def get_architectural_guidelines(componentName: str, location: str) -> str:
    """Returns architectural guidelines for a React project.

    Args:
        componentName: component name
        location: URL of the architectural guidelines
    """
    
    print(f"Tool call: Getting architectural guidelines for {componentName} from {location}")
    response = requests.get(location)
    html_content = response.text
    return html2text.html2text(html_content)

@tool
def get_ux_styleguide(componentName: str) -> str:
    """Returns the User Experience (UX) design style guide for a React project.

    Args:
        componentName: component name
    """
    
    print(f"Tool call: Getting UX style guide for {componentName}")
    with open("data/styleguide.md", "r") as file:
        response = file.read()
    return response

@tool
def get_business_rules(componentName: str) -> str:
    """Returns the Business Rules for a React project from a User Story.

    Args:
        componentName: component name
    """
    
    print(f"Tool call: Getting Business Rules for {componentName}")
    with open("data/userstory.md", "r") as file:
        response = file.read()
    return response

tools = [get_architectural_guidelines, get_ux_styleguide, get_business_rules]
llm_with_tools = llm.bind_tools(tools)

In [7]:
# Step x. define the chatbot
def coding_assistant(state: State):
    state.update({"num_called": state.get("num_called", 0) + 1})
    return {"messages": [llm_with_tools.invoke(state["messages"])], "num_called": state.get("num_called", 1)}

In [ ]:
# Step x. add the node to the graph
graph_builder.add_node("coding_assistant", coding_assistant)

In [ ]:
import json

class BasicToolNode:
    """A node that runs the tools requested in the last AIMessage."""

    def __init__(self, tools: list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        if messages := inputs.get("messages", []):
            message = messages[-1]
        else:
            raise ValueError("No message found in input")
        outputs = []
        for tool_call in message.tool_calls:
            tool_result = self.tools_by_name[tool_call["name"]].invoke(
                tool_call["args"]
            )
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call["name"],
                    tool_call_id=tool_call["id"],
                )
            )
        return {"messages": outputs}

tool_node = BasicToolNode(tools)
graph_builder.add_node("tools", tool_node)

In [ ]:
# Step x. add the start and end nodes
from typing import Literal
from langgraph.graph import START, END

def route_tools(
    state: State,
) -> Literal["tools", "__end__"]:
    """
    Use in the conditional_edge to route to the ToolNode if the last message
    has tool calls. Otherwise, route to the end.
    """
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return "__end__"


# The `tools_condition` function returns "tools" if the chatbot asks to use a tool, and "__end__" if
# it is fine directly responding. This conditional routing defines the main agent loop.
graph_builder.add_conditional_edges(
    "coding_assistant",
    route_tools,
    # The following dictionary lets you tell the graph to interpret the condition's outputs as a specific node
    # It defaults to the identity function, but if you
    # want to use a node named something else apart from "tools",
    # You can update the value of the dictionary to something else
    # e.g., "tools": "my_tools"
    {"tools": "tools", "__end__": "__end__"},
)

graph_builder.add_edge("tools", "coding_assistant")
graph_builder.add_edge(START, "coding_assistant")

In [11]:
# Step x. set the configuration needed for the state
config = {"configurable": {"thread_id": "1"}}

from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [12]:
# Step x. compile the graph
graph = graph_builder.compile(checkpointer=memory)

In [ ]:
# Step x. display the graph image
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [14]:
# Step x. define the stream function
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage

system_prompt = "You are an expert React developer."
def stream_graph_updates(user_input: str):
    result = graph.invoke({"messages": [SystemMessage(content=system_prompt), HumanMessage(content=user_input)]}, config)
    return result.get("messages", [])[-1].content

In [ ]:
import panel as pn  # GUI
pn.extension()
panels = [] # collect display

In [16]:
inputStyles = {
    'color': '#666',
    'font-size': '16px',
    'border-radius': '5px',
    'padding': '2px 2px',
    'margin': '0px 10px 0px 5px',
}

prompt_input = pn.widgets.TextInput(value="", placeholder='Enter text here…', width=600, styles=inputStyles)
chat_button = pn.widgets.Button(name="Chat!", styles=inputStyles)

In [17]:
userStyles = {
    'color': '#ffa8db',
    'font-size': '14px',
    'font-family': 'Inter, Arial, sans-serif',
    'background-color': '#333', 'border': '1px solid #999',
    'border-radius': '5px', 
    'padding': '2px 10px',
    'width': '90%'
}

assistantStyles = {
    'color': '#11fa00',
    'font-size': '14px',
    'font-family': 'Inter, Arial, sans-serif',
    'background-color': '#333', 'border': '1px solid #999',
    'border-radius': '10px', 'padding': '2px 10px',
    'width': '90%'
}

import html
def escape_html(text):
    return html.escape(text)

def collect_messages(_):
    prompt = prompt_input.value_input
    snapshot = graph.get_state(config)
    num = snapshot.values.get("num_called", 1)
    assistantLabel = "Assistant (called " + str(num) + " times):"

    if prompt != "":
        response = stream_graph_updates(prompt)
        panels.append(
            pn.Row('User:', pn.pane.HTML("<p>" + escape_html(prompt) + "</p>", width=600, styles=userStyles)))
        panels.append(
            pn.Row(assistantLabel, pn.pane.Markdown(response, styles=assistantStyles)))
    prompt_input.value = ""
    return pn.Column(*panels)

In [ ]:
interactive_conversation = pn.bind(collect_messages, chat_button)
dashboard = pn.Column(
    pn.Row(prompt_input, chat_button),
    pn.Row(interactive_conversation),
)
dashboard